# NPL - PROCESSAMENTO DE LINGUAGEM NATURAL

In [ ]:
# PARA TRABALHAR COM O GOOGLE COLAB É NECESSÁRIO:
# !apt-get update -qq
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
# !tar xf spark-3.1.2-bin-hadoop2.7.tgz
# !pip install -q findspark

import os
import findspark

os.environ["SPARK_HOME"] = "D:/spark-3.5.5-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

import pandas as pd

### Obs: para este projeto estamos trabalhando com o google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
SPARK = spark.read.csv('/content/drive/ml/imdb-reviews-pt-br.csv', escape='\"', inferSchema=True, header=True)

In [ ]:
# LEMATIZAÇÃO E STEMING (REMOÇÃO DE PALAVRAS DESNECESSARIAS PARA O PROCESSAMENTO COMO PRONOMES, PREPOSIÇÕES "STOP-WORDS", ETC)
# PARA MELHORAR O RECURSO VISUAL UTILIZAREMOS O WORDCLOUD, DESSA MANEIRA VAMOS ENXERGAR QUAIS AS PALAVRAS MAIS UTILIZADAS EM UM TEXTO
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# INICAMOS PEGANDO APENAS UMA AMOSTRA DE 10% PARA AJUDAR O PROCESSAMENTO (collect é mt pesado)
AMOSTRA = SPARK.select('text_pt').sample(fraction=0.1, seed=101)
DATA = [texto['text_pt'] for texto in AMOSTRA.collect()]

wordcloud = WordCloud(collocations=False, 
                      prefer_horizontal=1,
                      width=1000,
                      height=600).generate(str(DATA))

plt.figure(figsize=(20,8))
plt.imgshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# REMOÇÃO DE CARACTERES ESPECIAIS
import string
print(string.punctuation)

SPARK = SPARK.withColumn('texto_regex', f.regexp_replace('text_en', "[\$#,\"!%&'()*+,-./:;<=>?@[\]^_`{|}~\\\\]"), "")
SPARK = SPARK.withColumn('texto_limpo', f.trim(SPARK.texto_regex))

In [ ]:
# TOKENIZAÇÃO DO TEXTO
# UM TEXTO QUALQUER ===>  [UM][TEXTO][QUALQUER]
# TEXTO COMPOSTO GUARDA-CHUVA ====> [TEXTO][COMPOSTO][GUARDA-CHUVA]
from pyspark.ml.features import Tokenizer

tokenizer = Tokenizer(inputCol='texto_limpo', outputCol='tokens')
SPARK_TOKENIZADO = tokenizer.transform(SPARK)

In [ ]:
from pyspark.sql.types import IntegerType

count_token = f.udf(lambda x: len(x), IntegerType())
SPARK_TOKENIZADO.select('texto_limpo', 'tokens').withColumn('qtd_tokens', count_token(f.col('tokens')))

In [ ]:
# REMOVENDO AS STOP-WORDS QUE APARECEM NA WORD CLOUD
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# stop_A = stopwords('portuguese')

from pyspark.ml.features import StopWordsRemover
# stop_B = StopWordsRemover.loadDefaultStopWords('portuguese')

remover = StopWordsRemover(inputCol='tokens', outputCol='texto_final')
SPARK_TOKENIZADO_WORDS = remover.transform(SPARK_TOKENIZADO)
SPARK_TOKENIZADO_WORDS.select('texto_limpo', 'tokens').withColumn('qtd_tokens', count_token(f.col('tokens')))\
                                                      .withColumn('qtd_tokens_limpos', count_token(f.col('texto_final')))

In [ ]:
#